In [11]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = "spark-3.2.0"
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [12]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-27 23:02:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.36MB/s    in 0.2s    

2021-11-27 23:02:01 (5.36 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [13]:
# Spark
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [14]:
# Imports
from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date


In [15]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
file="amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("delimiter", "\t").option("header","True").option("inferSchema","True").csv(SparkFiles.get(file))

# Show DataFrame
df.show(truncate=True)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [16]:
# count the number of records in the dataset
df.count()

348657

In [17]:
### Get count of null values in pyspark
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               6|          6|            6|          6|   6|                6|              9|         40|         20|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [18]:
# drop rows with missing values
df=df.dropna()

In [19]:
# check if values dropped
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               0|          0|            0|          0|   0|                0|              0|          0|          0|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [20]:
# count how many rows now
df.count()

348613

In [21]:
#drop any duplicate rows
df = df.dropDuplicates()

In [22]:
# count rows again after dropping duplicate rows
df.count()

348613

In [ ]:
# check column data types
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [ ]:
# change review_date column from string to date
df = df.withColumn('review_date',to_date(df.review_date, 'yyyy-mm-dd'))

In [ ]:
# change star_rating column from string to integer
df = df.withColumn("star_rating",df["star_rating"].cast(IntegerType()))


In [ ]:
# check to see if types changed
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [ ]:
# check dataframe
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-01-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [ ]:
#create review df
review_id = df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
review_id.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R9CO86UUJCAW5|   40884699|B00VGTN02Y|     786681372| 2015-01-31|
|R3PR8X6QGVJ8B1|   23208852|B005KIWL0E|     618251799| 2015-01-31|
|R39BO2819ABUPF|   17100246|B007UNSHJ6|     810480328| 2015-01-31|
|R3ADL7V6EGGEEP|   13533670|B00WHFNXB4|      20765193| 2015-01-31|
|R1OXYPBPLVRMI5|   38541771|B000M5NBYU|     812890709| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
#create products df
products = df.select("product_id", "product_title")
products = products.dropDuplicates(["product_id"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1933438304|Penguin Tote: I'd...|
|3924862206|Kukishinden Ryû (...|
|8862938861|Moleskine Luggage...|
|886293887X|Moleskine 3.75x2....|
|8866138274|Moleskine Luggage...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# create customers df 
customers = df.groupBy("customer_id").count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42596797|             1|
|   12296525|             1|
|   49101505|             1|
|   28377689|             1|
|   14468845|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:<endpoint>:<port>/<db_name>"
config = {"user":"<user>", 
          "password": "<pass>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id DataFrame to review_id_table table in RDS

review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products DataFrame to products table in RDS

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers DataFrame to customers table in RDS

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)